In [ ]:
!pip install astroNN

     |████████████████████████████████| 9.3MB 6.6MB/s 
     |████████████████████████████████| 6.5MB 22.2MB/s 
     |████████████████████████████████| 51kB 7.7MB/s 
     |████████████████████████████████| 3.2MB 61.4MB/s 
  Created wheel for astroNN: filename=astroNN-1.0.1-cp37-none-any.whl size=9284594 sha256=5ad2318108d76c2a24c30a884235887ff4e88fb587c6679a227a0c2f6895d766
  Stored in directory: /root/.cache/pip/wheels/22/48/89/bcbab7475e4bff84e86a914145ea3ba907b6afb63f9fbe04b0
  Created wheel for astroquery: filename=astroquery-0.4.1-cp37-none-any.whl size=3831874 sha256=c05f2f01dd3ac63ce00bc2b0115443da7c56092701fe72f377943bdb70033e35
  Stored in directory: /root/.cache/pip/wheels/88/f8/b7/a254cd96e808f708bc0b7d755a8e095c56fbbe94099d7b464f
Successfully built astroNN astroquery


In [ ]:
#importing required libraries
import numpy as np
import keras,os
import seaborn as sns

from keras.models import Sequential
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
from keras.datasets import cifar10
from astroNN.datasets import galaxy10
from tensorflow.keras import utils

WARNING! APOGEE environment variable SDSS_LOCAL_SAS_MIRROR not set
WARNING! Gaia environment variable GAIA_TOOLS_DATA not set


In [ ]:
#loading the dataset
images, labels = galaxy10.load_data()

Galaxy10.h5:  99%|█████████▉| 209M/210M [00:06<00:00, 52.6MB/s]

Downloaded Galaxy10 successfully to /root/.astroNN/datasets/Galaxy10.h5


Galaxy10.h5: 210MB [00:06, 30.7MB/s]                           


In [ ]:
# To convert the labels to categorical 10 classes
labels = utils.to_categorical(labels, 10)

#converting the arrays to float type
labels = labels.astype(np.float32)
images = images.astype(np.float32)

In [ ]:
labels[0]

array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [ ]:
X = images
y = labels

In [ ]:
#splitting the data into train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

In [ ]:
#printing the shapes of test and train data
print('Size of X_train', X_train.shape)
print('Size of y_train', y_train.shape)
print('Size of X_test', X_test.shape)
print('Size of y_test', y_test.shape)

Size of X_train (19606, 69, 69, 3)
Size of y_train (19606, 10)
Size of X_test (2179, 69, 69, 3)
Size of y_test (2179, 10)


#Building the VGG16 Model

In [ ]:
model = Sequential()
#first layer
model.add(Conv2D(input_shape=(69,69,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

#second layer
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

#third layer
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

#fourth layer
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

#fifth layer
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

#dense layer
model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=10, activation="softmax"))

opt = Adam(lr=0.001)
model.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 69, 69, 64)        1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 69, 69, 64)        36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 34, 34, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 128)       147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 17, 17, 256)       2

In [ ]:
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, 
                             save_best_only=True, save_weights_only=False, mode='auto', period=1)

early = EarlyStopping(monitor='val_acc', min_delta=0, patience=20, verbose=1, 
                      mode='auto')

#Training

In [ ]:
hist = model.fit(X_train, y_train, steps_per_epoch=50, validation_steps=10,epochs=5,callbacks=[checkpoint,early])

Epoch 1/5
 2/50 [>.............................] - ETA: 48:30 - loss: 193.9420 - accuracy: 0.2907